In [192]:
from selenium.webdriver import Chrome 
from bs4 import BeautifulSoup
import pymongo
import datetime
import time 
import pandas as pd
import re

In [2]:
url = "https://tabs.ultimate-guitar.com/tab/bob_dylan/as_i_went_out_one_morning_chords_927691"

In [3]:
browser = Chrome()

In [4]:
browser.get(url)

In [5]:
time.sleep(10)

In [6]:
html = browser.page_source

In [7]:
mc = pymongo.MongoClient()

In [8]:
db = mc['chordify']

In [9]:
raw_html = db['raw_html']

In [10]:
raw_html.insert_one(
    {'url': url,
     'datetime': datetime.datetime.now(),
     'html': html 
    })

In [120]:
sel = "section._3TQ4n._3zQIL"
body_sel = browser.find_element_by_css_selector(sel)

In [121]:
body_sel.text

'Capo on 4th Fret\n\nDm                 C                      Dm   C         Dm\nAs I went out one morning, to breathe the air around Tom Paine\'s\n    Dm               C            Dm      C       Dm\nI spied the fairest damsel, that ever did walk in chains\n    F              Am       Dm    C       Dm\nI offer\'d her my hand, she took me by the arm\n    Dm            C            Dm      C    Dm\nI knew that very instant, she meant to do me harm\n\n\nDm                 C             Dm       C     Dm\nDepart from me this moment, I told her with my voice\n Dm                       C       Dm            C        Dm\nSaid she, "But I don\'t wish to, said I, but you have no choice\nF                 Am         Dm                C       Dm\nI beg you, sir, she pleaded, from the corners of her mouth\nDm                C               Dm        C         Dm\nI will secretly accept you, and together we\'ll fly south\n\n\nDm                     C               Dm        C         Dm\nJust th

In [204]:
body_sel.text

ConnectionResetError: [Errno 54] Connection reset by peer

In [125]:
chords_sel = browser.find_elements_by_css_selector('span._3L0Da._3f5kI')
chords_sel[0]

<selenium.webdriver.remote.webelement.WebElement (session="c7e7021899be7bcd6c89683f09aa59db", element="0.22856330311087558-3")>

In [126]:
chords_sel[0].text

'Dm'

In [19]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [128]:
song_body = soup.select_one('pre._1YgOS')
song_body

<pre class="_1YgOS" style='font-size: 13px; font-family: "Roboto Mono", monospace;'>Capo on 4th Fret

<span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>                 <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>                      <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>   <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>         <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>
As I went out one morning, to breathe the air around Tom Paine's
    <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>               <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>            <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>      <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>       <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>
I spied the fairest damsel, that ever did walk in chains
    <span class="_3L0Da

In [129]:
lines = str(song_body).split('\n')
lines

['<pre class="_1YgOS" style=\'font-size: 13px; font-family: "Roboto Mono", monospace;\'>Capo on 4th Fret',
 '',
 '<span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>                 <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>                      <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>   <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>         <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>',
 "As I went out one morning, to breathe the air around Tom Paine's",
 '    <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>               <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>            <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>      <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>       <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Dm</span>',
 'I spied the fairest damsel, that ever did walk in chains

In [112]:
def strip_html(text):
    result = []
    in_tag = False
    for char in text:
        if char == '<':
            in_tag = True
        if not in_tag:
            result.append(char)
        if char == '>':
            in_tag = False
    return ''.join(result)

In [116]:
def parse_lines(lines):
    parsed_lines = []
    for line in lines: 
        if '_3L0Da' in line:
            parsed_lines.append({'chords': strip_html(line)})
        elif line and strip_html(line) == line:
            parsed_lines[-1]['words'] = line
    return parsed_lines 

In [131]:
parse_lines(lines)

[{'chords': 'Dm                 C                      Dm   C         Dm',
  'words': "As I went out one morning, to breathe the air around Tom Paine's"},
 {'chords': '    Dm               C            Dm      C       Dm',
  'words': 'I spied the fairest damsel, that ever did walk in chains'},
 {'chords': '    F              Am       Dm    C       Dm',
  'words': "I offer'd her my hand, she took me by the arm"},
 {'chords': '    Dm            C            Dm      C    Dm',
  'words': 'I knew that very instant, she meant to do me harm'},
 {'chords': 'Dm                 C             Dm       C     Dm',
  'words': 'Depart from me this moment, I told her with my voice'},
 {'chords': ' Dm                       C       Dm            C        Dm',
  'words': 'Said she, "But I don\'t wish to, said I, but you have no choice'},
 {'chords': 'F                 Am         Dm                C       Dm',
  'words': 'I beg you, sir, she pleaded, from the corners of her mouth'},
 {'chords': 'Dm       

In [139]:
first_line = parsed_lines[0]
first_line

{'chords': 'Dm                 C                      Dm   C         Dm',
 'words': "As I went out one morning, to breathe the air around Tom Paine's"}

In [157]:
parsed_lines[0]['chords']

'Dm                 C                      Dm   C         Dm'

In [152]:
parsed_lines[0]['words'][19]

'o'

In [170]:
c = parsed_lines[0]['chords']
for i, chord in enumerate(c):
    if c[i] != c[-1]:
        if c[i+1] == ' ':
            chord = chord
        else:
            chord = c[i] + c[i+1] 

In [171]:
parsed_lines[0]['chords']

'Dm                 C                      Dm   C         Dm'

In [174]:
parsed_lines[0]

{'chords': 'Dm                 C                      Dm   C         Dm',
 'words': "As I went out one morning, to breathe the air around Tom Paine's"}

In [292]:
# chord_lyric_dict = {}

# def get_chords(pl):
#     chord_idxs = []
#     chords = []
#     for i, chord in enumerate(pl['chords']):
#         if pl['chords'][i] != pl[-1]:
#             if c[i+1] == ' ':
#                 chord = chord
#             else:
#                 chord = c[i] + c[i+1] 
#         chord_idxs.append(i)
#         chords.append(chord)
#     return chord_idxs, chords 

def get_chords(pl):
    chord_idxs = []
    chords = []
    c_string = pl['chords']
    for chord in re.finditer('\w+', c_string):
        chord_idxs.append(chord.start())
        chords.append(chord.group())
    return chord_idxs, chords 

def get_words(pl):
    word_idxs = []
    words = []
    w_string = pl['words']
    for word in re.finditer('\w+', w_string):
        word_idxs.append(word.start())
        words.append(word.group())
    return word_idxs, words

In [ ]:
def match_chords_words(pl):
    for w in word_idxs:
        for c in chord_idxs:
            if w

In [293]:
get_chords(parsed_lines[0])

([0, 19, 42, 47, 57], ['Dm', 'C', 'Dm', 'C', 'Dm'])

In [294]:
get_words(parsed_lines[0])

([0, 3, 5, 10, 14, 18, 27, 30, 38, 42, 46, 53, 57, 63],
 ['As',
  'I',
  'went',
  'out',
  'one',
  'morning',
  'to',
  'breathe',
  'the',
  'air',
  'around',
  'Tom',
  'Paine',
  's'])

In [277]:
parsed_line = parsed_lines[0]
parsed_line

{'chords': 'Dm                 C                      Dm   C         Dm',
 'words': "As I went out one morning, to breathe the air around Tom Paine's"}